# Optimización de Consultas

creamos la sesión de Spark

In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

val spark: SparkSession = 
    NotebookSparkSession
      .builder()
      .appName("Queries Optimization")
      .master("local[*]")
      .getOrCreate()


Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.9.1/almond-spark_2.12-0.9.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.5/spark-sql_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.5/spark-parent_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-hadoop/1.10.1/parquet-hadoop-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-vector/0.10.0/arrow-vector-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.8/scala-library-2.12.8.pom
Downloaded https://repo1.maven.org/maven2/org/spark-project/spark/unused/1.0.0/unused-1.0.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.5/spark-catalyst_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.

Downloaded https://repo1.maven.org/maven2/org/codehaus/janino/commons-compiler/3.0.9/commons-compiler-3.0.9.pom
Downloaded https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.5.0/protobuf-java-2.5.0.pom
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.20.v20190813/jetty-server-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-core/2.7.9/jackson-core-2.7.9.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-shims/1.5.5/orc-shims-1.5.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-unsafe_2.12/2.4.5/spark-unsafe_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.7/antlr4-runtime-4.7.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-memory/0.10.0/arrow-memory-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.pom
Downloaded https://repo1.maven.org/maven2

Downloaded https://repo1.maven.org/maven2/oro/oro/2.0.8/oro-2.0.8.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/jcl-over-slf4j/1.7.16/jcl-over-slf4j-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/net/sf/py4j/py4j/0.10.7/py4j-0.10.7.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-json/3.1.5/metrics-json-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/core/jersey-common/2.22.2/jersey-common-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-format/2.4.0/parquet-format-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-shuffle_2.12/2.4.5/spark-network-shuffle_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-common_2.12/2.4.5/spark-network-common_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/com/carrotsearch/hppc/0.7.2/hppc-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-s

Logging

In [2]:
import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}
Logger.getRootLogger().setLevel(Level.ERROR)

import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


imports

In [ ]:
import spark.implicits._
import spark.sqlContext.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

# Los Datos

El dataset ha sido obtenido de:
https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide

En el se observan los casos diarios de Covid-19 por país hasta el 14-12-20

En la segunda parte se utilizan los datos de las medidas aplicadas a cada país por fecha de inicio y fin:

https://www.ecdc.europa.eu/en/publications-data/download-data-response-measures-covid-19

La última consulta para calcular las infecciones por km2:

https://www.kaggle.com/tanuprabhu/population-by-country-2020

## Creo una clase para trabajar con infecciones 

In [60]:
case class Infection(day : Int, 
                     month : Int, 
                     year : Int, 
                     nCases: Int, 
                     nDeaths : Int, 
                     country : String,  
                     continent : String) 
extends Serializable

defined class Infection

Y un método para medir tiempos de ejecución

In [ ]:
def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

# Empiezo trabajando con RDDs

In [ ]:
val readFromFile = spark.sparkContext.textFile("data.csv")

Creo una funcion para trabajar con un RDD de infecciones

In [ ]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

Calculo la media de infecciones diarias por país

In [ ]:
  def infectionGrowthAverage(infections : RDD[Infection]) : RDD[(String, Int)]= {

    val countriesAndCases : RDD[(String, Iterable[Int])] = 
      infections.map(x => (x.country,x.nCases))
      .groupByKey()
      
    countriesAndCases.mapValues(x => (x.sum / x.size)).sortBy(_._2)
  }

Muestro el resultado y el tiempo de ejecución

In [ ]:
val infectionRDD = infections(readFromFile)

run(infectionGrowthAverage(infectionRDD))
.foreach(println)

# Hago los mismos calculos con un DataFrame

Convierto el RDD obtenido previamente en un DataFrame para inferir la clase infección

In [ ]:
val infectionDF = spark.createDataFrame(infectionRDD)


Utilizo los métodos de la clase DF que incluye uno optimizado para calcular la media.

Ejecuto y comprabamos como el tiempo de ejecución es significativamente menor que en RDD

In [ ]:
run(infectionDF.
    groupBy("country")
    .avg("nCases")
    .orderBy("avg(nCases)")
    .count
)

Otra opción es crear el DataFrame directamente importando los datos pero deja de ser un DF de infecciones

In [ ]:
val dfCovid = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("covidworldwide.csv")

In [ ]:
dfCovid.schema

In [ ]:
dfCovid.explain

In [ ]:
run(dfCovid.toDF.groupBy("countriesAndTerritories")
    .agg(mean("cases")).orderBy("avg(cases)")).show(2000)

puedo definir el esquema manualmente para crear el DataFrame

In [ ]:
//Defino el esquema manualmente pero podría verlo importando el csv y viendo como lo hace de base spark

val schema = new StructType()
    .add("dateRep",StringType,true)
    .add("day",IntegerType,true)
    .add("month",IntegerType,true)
    .add("year",IntegerType,true)
    .add("cases",IntegerType,true)
    .add("deaths",IntegerType,true)
    .add("countriesAndTerritories",StringType,true)
    .add("geoId",StringType,true)
    .add("countryterritoryCode",StringType,true)
    .add("popData2018",IntegerType,true)
    .add("continentExp",StringType,true)

In [ ]:
val df = spark.read
.format("csv")
.option("header","true")
.schema(schema)
.load("data.csv")

In [ ]:
df.printSchema()

In [ ]:
df.show()

# Y con un DataSet

In [ ]:
val infectionDS = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.csv("covidworldwide.csv")
.as[(String,String,String,String,String,String,String,String,String,String,String,String)]

In [ ]:
run(
    infectionDS.groupBy($"countriesAndTerritories")
    .agg(avg($"cases").as[Double])
    .orderBy("avg(cases)")
    .count
    )

### Otras opciones menos eficientes
(Work in progress)

In [ ]:
infectionDS.groupByKey(p => p._7) //hace shuffling de los datos
        .mapValues(p => p._5.toDouble)
        .mapGroups((k,vs) => (???))

In [ ]:
infectionDS.groupByKey(p => p._7) //mas eficiente con el reduce
        .mapValues(p => p._5.toDouble)
        .reduceGroups((acc,str)=> ???)


In [ ]:
val infGrowAvg = new Aggregator[]{ //utilizando un Aggregator
    
}.toColumn 

### Intento trabajar con DataSet de infecciones Dataset[Infection] pero falla 
(Work in progress)

Esto nos dará error pues no se pueden crear datasets sin tipo de datos y no entiende las infecciones

In [ ]:
val infectionDS = spark.createDataset(infections(readFromFile)).as[Infection]

In [ ]:
val ds : Dataset[Infection] = spark.createDataset(infectionRDD).as[Infection]

Necesito importar los Encoders y explicitar el tipo de datos

In [ ]:
import org.apache.spark.sql.Encoders
Encoders.product[Infection]

In [ ]:
import org.apache.spark.sql.{Encoder, Encoders}
import org.apache.spark.sql.Encoders

val dataset = spark.createDataset(infectionRDD)(Encoders.product[Infection])

(A partir de aquí intento crear un DataSet de infecciones pero siempre obtengo el mismo error)

In [ ]:
val infectionDS = spark.createDataset(infectionRDD).as[Infection]

In [ ]:
  def infections(lines : Dataset[(String,String,String,String,String,String,String,String,String,String,String,String)]) 
                       : Dataset[Infection] = 
    lines.map(line => {
      Infection(
        day = line._1.toInt,
        month = line._2.toInt,
        year = line._3.toInt,
        nCases = line._4.toInt,
        nDeaths = line._5.toInt,
        country = line._6,
        continent = line._10
      )
    })

In [ ]:
infections(infectionDS).as[Infection]

# Utilizo una segunda tabla y cruzo datos con RDD, DS y DF
(De momento solo uso DataFrames)

In [ ]:
val dfMeasures = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("response_graphs_data_2021-04-15.csv")
dfMeasures.show
dfMeasures.schema

In [ ]:
val dfPopulation = spark.read
.option("header", "true")
.option("charset", "UTF8")
.option("delimiter",",")
.option("inferSchema", "true")
.csv("population_by_country_2020.csv")
dfPopulation.show
dfPopulation.schema

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [ ]:
val dfCovidClean = dfCovid.select($"*",$"dateRep",translate($"dateRep","/","-").as("new-date")).drop("dateRep").show()

Hago una consulta de prueba para obtener la media solo de los casos en España

In [ ]:
val spainCovid = dfCovid.select("dateRep","cases").where("countriesAndTerritories == 'Spain'").toDF

In [ ]:
run(spainCovid.agg(avg("cases"))).show

Cruzo los datos con un Join y hago algunas consultas sencillas

In [ ]:
val megaDF = dfCovid.join(dfMeasures, $"Country" === $"countriesAndTerritories").schema

In [ ]:
megaDF.select("cases","deaths","dateRep","Response_measure")
    .where("countriesAndTerritories == 'Spain'").show

In [ ]:
run(dfCovid.join(dsMeasures, $"Country" === $"countriesAndTerritories")
        .select("cases","deaths","dateRep","Response_measure")
        .where("countriesAndTerritories == 'Spain'"))

## Creo una consulta para calcular la media de infecciones por Km2

In [ ]:
run(
dfCovid.join(dfPopulation, $"Country (or dependency)" === $"countriesAndTerritories")
        .select($"Country (or dependency)" as "Country",
                $"dateRep" as "date",
                $"cases",
                $"Land Area (Km\u00b2)",
                $"cases" / $"Land Area (Km\u00b2)" as "infection Per Km\u00b2")
        .groupBy("Country")
        .avg("infection Per Km\u00b2")
        .orderBy(asc("avg(infection Per Km²)"))
        .count
    )

# Notas y observaciones personales interesantes 

In [ ]:
dfCovid.select("dateRep","countriesAndTerritories","cases").show //aplico los métodos de DFs

In [ ]:
dfCovid.createOrReplaceTempView("covid") //se pueden aplicar consultas SQL sobre DF

spark.sql("SELECT * FROM covid WHERE countriesAndTerritories == 'Spain'").show

In [ ]:
//filtrados sobre dataframes
dfCovid.filter($"continentExp" === "Asia" || $"continentExp" === "Europe").sort($"continentExp".asc).show()